# 🧠 Palabras Encadenadas: Reconocimiento y Síntesis de Voz

En este juego, la máquina y el usuario se turnan para decir palabras que deben comenzar por la última letra de la palabra anterior.

Hemos utilizado:
- **Faster Whisper (ASR)**: para reconocimiento automático de voz en español.
- **MMS-TTS (Facebook, 2023)**: para síntesis de voz natural en español.
- **Google Colab + Navegador**: para grabación de audio y reproducción directa.

El código está basado en los cuadernos THLN_TTS.ipynb y THLN_ASR.ipynb.

### 🚀 ¿Cómo funciona?
1. La máquina comenzará con una palabra **aleatoria**.
2. El usuario deberá responder con una palabra que empiece por la última letra de la palabra generada.
3. La IA revisará si la palabra es válida y responderá con otra palabra correcta.
4. El juego continúa hasta que:
   - El usuario repita una palabra.
   - No se encuentra la palabra escuchada en el diccionario.
   - No se detecte audio.

# Instalaciones

Comienzamos instalando todas las librerías necesarias para el reconocimiento de voz (ASR), la síntesis de voz (TTS) y el manejo de audios.

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install scipy
!pip install --upgrade -q faster-whisper ipython-autotime
!pip install soundfile
!wget https://raw.githubusercontent.com/words/an-array-of-spanish-words/master/index.json -O palabras.json

Luego, importamos las librerías que usaremos a lo largo del cuaderno, incluyendo los modelos de ASR y TTS, así como otras herramientas necesarias.

In [ ]:
# ===============================
# 2. IMPORTACIÓN DE LIBRERÍAS
# ===============================

from faster_whisper import WhisperModel
from transformers import VitsModel, VitsTokenizer
import soundfile as sf
from scipy.io.wavfile import write
import os
import torch
from IPython.display import Audio, display
import re
import json
import random
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

Aquí cargamos el listado de palabras desde un archivo JSON que usaremos como diccionario, luego las convertimos a minúscula para evitar problemas y las almacenamos en una lista.

Para que el juego sea diferente en cada sesión, creamos una función que selecciona una palabra aleatoria del diccionario de palabras que hemos cargado. Así, comenzaremos el juego siempre con una palabra distinta.

In [ ]:
# ===============================
# 3. CARGA DEL DICCIONARIO LOCAL
# ===============================

# Cargar el listado de palabras desde el archivo JSON
with open('palabras.json', 'r', encoding='utf-8') as file:
    diccionario_palabras = json.load(file)

# Convertir todo a minúsculas para evitar problemas de mayúsculas
# y mantenerlo como lista para poder usar random.choice
diccionario_palabras = list(set([palabra.lower() for palabra in diccionario_palabras]))

# Función para obtener una palabra aleatoria del diccionario
def obtener_palabra_aleatoria():
    return random.choice(diccionario_palabras)

En esta sección, cargamos los dos modelos principales:

- **Faster Whisper**: para transcribir lo que decimos en español.
- **MMS-TTS (Facebook)**: para que transcribir el texto a una voz natural en español.

Estos modelos nos permiten interactuar por voz de manera medianamente fluida (la transcripción de nuestro audio al texto es algo lenta).

In [ ]:
# ===============================
# 4. CARGA DE MODELOS ASR Y TTS
# ===============================

# Cargar el modelo Faster Whisper (ASR)
asr_model = WhisperModel(model_size_or_path="small", device="auto", compute_type="int8")

# Cargar el modelo MMS-TTS en español
modeloMMS = VitsModel.from_pretrained("facebook/mms-tts-spa")
tokenizerMMS = VitsTokenizer.from_pretrained("facebook/mms-tts-spa")

Aquí definimos las funciones necesarias para:
- **Transcribir audio a texto** (reconocimiento de voz).
- **Generar audio a partir de texto** (síntesis de voz).

Llamaremos luego a estas funciones en nuestro flujo principal del juego

In [ ]:
# ===============================
# 5. FUNCIONES ASR Y TTS
# ===============================


# Función de reconocimiento de voz
def transcribir_audio(audio_path):
    print("🎧 Transcribiendo tu palabra...")
    # Ejecutamos el método transcribe
    segmentos, info = asr_model.transcribe(audio_path, beam_size=1)
    texto = ""
    for segmento in segmentos:
        texto += segmento.text
    texto_completo = texto.strip().lower()
    texto_limpio = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', texto_completo)
    if not texto_limpio:
        print("⚠️ No se ha detectado texto.")
    else:
        print(f"📝 Has dicho: {texto_limpio}")
    return texto_limpio

# Función de síntesis de voz
def generar_audio(texto, ruta_salida="respuesta.wav"):
    print(f"🗣️ Generando respuesta:")

    token_ids = tokenizerMMS(texto, return_tensors="pt")
    input_ids = token_ids["input_ids"]


    with torch.no_grad():
        outputs = modeloMMS(input_ids)

    hablaES = outputs["waveform"]
    display(Audio(hablaES, rate=16000, autoplay=True))

Definimos las reglas que validan si una palabra es correcta, comprobando:
- Si empieza por la letra correcta.
- Si no ha sido utilizada antes.

Estas reglas garantizan las normas básicas del juego

In [ ]:
# ===============================
# 6. LÓGICA DE PALABRAS ENCADENADAS
# ===============================

# Lista para almacenar las palabras usadas
palabras_usadas = []

# Comprobar si una palabra existe en el diccionario
def palabra_existe(palabra):
    return palabra in diccionario_palabras

# Buscar una palabra válida que empiece por una letra dada
def buscar_palabra_por_letra(letra, usadas):
    opciones = [p for p in diccionario_palabras if p.startswith(letra) and p not in usadas]
    if opciones:
        return random.choice(opciones)  # Elegir una aleatoria
    else:
        return None  # No se encontró ninguna palabra

Definimos una función que permite grabar audio directamente desde el navegador, usando las capacidades de Google Colab y JavaScript. Así, el usuario podrá hablar y que el sistema le escuche, sin necesidad de instalar nada ni mayores complicaciones.

In [ ]:
# ===============================
# 7. FUNCIÓN PARA GRABAR AUDIO
# ===============================

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def graba(ficheroAudio, segundos=7):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (segundos*1000))
  b = b64decode(s.split(',')[1])
  with open(ficheroAudio,'wb') as f:
    f.write(b)
  return "Audio grabado en "+ficheroAudio

Esta es la lógica principal del juego:

- La máquina comienza diciendo una palabra aleatoria.
- El usuario debe decir una palabra válida.
- La IA responderá con una nueva palabra.
- El juego continuará hasta que se falle alguna regla.

¡Hora de jugar! 🕹️

In [ ]:
# ===============================
# 8. FLUJO PRINCIPAL DEL JUEGO
# ===============================

# Palabra inicial
palabra_actual = obtener_palabra_aleatoria()
palabras_usadas.append(palabra_actual)

# Primera respuesta
generar_audio(f"Empezamos el juego. Mi palabra es: {palabra_actual}")

# Bucle principal
while True:
    #### Turno del usuario
    print(f"\n👉 Tu turno. Di una palabra que empiece por '{palabra_actual[-1]}':")

    # Grabar audio del usuario
    graba("grabacion.wav")

    # Transcribir la palabra del usuario
    texto_transcrito = transcribir_audio("grabacion.wav")

    # Verificar si se ha detectado alguna palabra
    if not texto_transcrito:
      generar_audio("No te he oído. Inténtalo de nuevo más tarde")
      break  # Salir del bucle del juego

    # Tomar solo la primera palabra
    palabra_usuario = texto_transcrito.split()[0]

    # Validaciones
    if palabra_usuario in palabras_usadas:
        print("❌ Esa palabra ya fue usada. Intenta otra.")
        generar_audio("Esa palabra ya fue usada. Intenta otra.")
        continue

    if palabra_usuario[0] != palabra_actual[-1]:
        print(f"❌ La palabra no empieza por '{palabra_actual[-1]}'. Intenta otra.")
        generar_audio(f"La palabra no empieza por {palabra_actual[-1]}. Intenta otra.")
        continue

    # Palabra aceptada
    print(f"✅ Palabra aceptada: {palabra_usuario}")
    palabras_usadas.append(palabra_usuario)

    #### Turno de la máquina
    siguiente_letra = palabra_usuario[-1]
    palabra_ia = buscar_palabra_por_letra(siguiente_letra, palabras_usadas)

    if palabra_ia:
        palabras_usadas.append(palabra_ia)
        palabra_actual = palabra_ia   # Actualizar palabra actual
        generar_audio(f"Mi palabra es {palabra_ia}")
    else:
        generar_audio("No puedo encontrar esa palabra, hemos terminado")
        break  # Salir del juego si no encuentra la palabra